# Download End-of-Day NSE Stock Prices

In [1]:
# Import modules
# # pip install jugaad-data pandas
from nsepy import get_history
from datetime import datetime, timedelta
import pandas as pd
from jugaad_data.holidays import holidays
from jugaad_data.nse import bhavcopy_save
pd.core.common.is_list_like = pd.api.types.is_list_like #For solving import pandas_datareader issue
import numpy as np
import datetime
import csv
import requests
import pandas_datareader.data as web
import pandas_datareader as pdr
from pandas_datareader import data, wb
from datetime import date
#to plot within notebook
import matplotlib.pyplot as plt
%matplotlib inline

from dateutil.relativedelta import relativedelta

import  zipfile, os, io

import requests

In [2]:
path = r"A:\FinMinII\Trading\NSE_Indices\Backtest\bhavcopy"

# # # Input Start and End Date
start = datetime.datetime(2018,1,1) # keep the strt date starting from Friday for weekly analysis (Friday is first day for expirty week and month)
end = pd.to_datetime('today').normalize() #datetime.datetime(2022,2,2)

print(start)
print(end)

2018-01-01 00:00:00
2022-03-08 00:00:00


In [3]:
import os
os.getcwd()
os.chdir(path)
os.getcwd()

'A:\\FinMinII\\Trading\\NSE_Indices\\Backtest\\bhavcopy'

In [4]:

# date_range = pd.bdate_range(start, end , 
#                          freq='C', holidays = holidays(2020,12))

# #bdate = business days (weekends excluded by default)
# # start and end dates in "MM-DD-YYYY" format
# # holidays() function in (year,month) format
# #freq = 'C' is for custom

# print(date_range)

In [5]:
hd = []
for i in range (start.year, end.year+1):
    print(i)
    hd = hd+holidays(i) 


2018
2019
2020
2021
2022


In [6]:
Working_Dates = pd.bdate_range(start=start, end = end, 
                    freq='C', holidays = hd)

Working_Dates = [x.date() for x in Working_Dates]
#This is known as a list comprehension; let's print it out to see what happens

print(len(Working_Dates))

1032


In [7]:
# for dt in Working_Dates:
#     bhavcopy_save(dt, path)
    

# Capture all NSE Bhavcopy (EOD Stock Prices) Files into One File

In [8]:
import glob
List_of_Bhavcopies = glob.glob(os.path.join(path,'*bhav.csv'))

In [9]:
bhavcopy_df = pd.read_csv(List_of_Bhavcopies[0])

In [10]:
bhavcopy_df.columns

Index(['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'LAST', 'PREVCLOSE',
       'TOTTRDQTY', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES', 'ISIN',
       'Unnamed: 13'],
      dtype='object')

In [11]:
bhavcopy_df.drop(columns = ['Unnamed: 13'],inplace = True)

In [12]:

bhavcopy_df.info()
bhavcopy_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948 entries, 0 to 1947
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SYMBOL       1948 non-null   object 
 1   SERIES       1943 non-null   object 
 2   OPEN         1948 non-null   float64
 3   HIGH         1948 non-null   float64
 4   LOW          1948 non-null   float64
 5   CLOSE        1948 non-null   float64
 6   LAST         1948 non-null   float64
 7   PREVCLOSE    1948 non-null   float64
 8   TOTTRDQTY    1948 non-null   int64  
 9   TOTTRDVAL    1948 non-null   float64
 10  TIMESTAMP    1948 non-null   object 
 11  TOTALTRADES  1948 non-null   int64  
 12  ISIN         1948 non-null   object 
dtypes: float64(7), int64(2), object(4)
memory usage: 198.0+ KB


,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN
0,1003GS2019,GS,69.66,69.66,69.66,69.66,69.66,73.32,1,6.966000e+01,01-APR-2019,1,IN0020010065
1,20MICRONS,EQ,39.45,40.55,38.05,40.15,40.10,38.20,70139,2.786821e+06,01-APR-2019,401,INE144J01027
2,21STCENMGM,EQ,19.60,19.60,19.60,19.60,19.60,19.25,3415,6.693400e+04,01-APR-2019,15,INE253B01015
3,3IINFOTECH,EQ,3.85,3.90,3.80,3.80,3.85,3.85,1259366,4.840342e+06,01-APR-2019,421,INE748C01020
4,3MINDIA,EQ,24270.00,24800.00,24099.75,24294.75,24195.00,24260.85,4621,1.133018e+08,01-APR-2019,1150,INE470A01017


In [13]:
# bhavcopy_df.columns = bhavcopy_df.columns.str.replace(' ', '')
# #we can only remove whitespace from 'object' datatype
# # Hence the if condition in the below otherwise you will get an error
# bhavcopy_df = bhavcopy_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

In [14]:
# #Converting date column to datetime dtype
# bhavcopy_df['TIMESTAMP'] = pd.to_datetime(bhavcopy_df['TIMESTAMP'])

# #Setting DATE1 column as index
# bhavcopy_df.set_index(['TIMESTAMP'], inplace=True)

# #Filtering only for EQ, BE & SM series.
# new_bhavcopy_df = bhavcopy_df[bhavcopy_df['SERIES'].isin(['EQ', 'BE', 'SM'])]

# #Grabbing the first 5 rows of the new_df
# new_bhavcopy_df.head()

In [15]:
final_df = pd.DataFrame() #empty dataframe

for csv_file in List_of_Bhavcopies:
    df = pd.read_csv(csv_file)
    df.columns = df.columns.str.replace(' ', '')
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    df.set_index(['TIMESTAMP'], inplace=True)

    if 'Unnamed:13' in df.columns:
        df.drop(['Unnamed:13'], axis=1, inplace=True)

    df_trim = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

    new_df = df_trim[df_trim['SERIES'].isin(['EQ', 'BE', 'SM'])]
    final_df = final_df.append(new_df)

final_df.sort_index(inplace=True) #to sort by dates

In [16]:
final_df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,ISIN
TIMESTAMP,,,,,,,,,,,,
2018-01-01,20MICRONS,EQ,58.95,60.7,57.9,58.75,58.70,58.30,156506,9.316661e+06,1527,INE144J01027
2018-01-01,PILANIINVS,BE,3210.00,3248.0,3170.0,3178.75,3170.00,3186.65,559,1.795369e+06,74,INE417C01014
2018-01-01,PIIND,EQ,968.80,968.8,952.0,958.65,956.35,962.50,15293,1.469429e+07,1114,INE603J01030
2018-01-01,PIDILITIND,EQ,904.95,908.8,898.0,900.00,900.80,902.20,160477,1.448900e+08,5062,INE318A01026
2018-01-01,PHOENIXLTD,EQ,649.80,664.4,627.0,633.65,631.00,626.95,136550,8.903390e+07,4562,INE211B01039
